In [1]:
## mmrotate1.x기반으로 진행 
# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)
# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.0.0rc1
3.1.0
11.1
GCC 7.3


In [2]:
## config LOAD
from mmengine.config import Config
cfg = Config.fromfile('custom_CONFIG_HJS_VER1.py')

In [3]:
from mmdet.apis import init_detector, inference_detector
import mmrotate

checkpoint_file = 'ver1/epoch_2790.pth'
model = init_detector(cfg, checkpoint_file, device='cuda:0')

Loads checkpoint by local backend from path: ver1/epoch_2790.pth


In [4]:
import json
import glob as _glob
import os
import random
import math
import numpy as np
import csv

def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches

csv.register_dialect(
    'mydialect',
    delimiter = ',',
    quotechar = '"',
    doublequote = True,
    skipinitialspace = True,
    lineterminator = '\r\n',
    quoting = csv.QUOTE_MINIMAL)

def writecsv(csvname,contents):
    f = open(csvname, 'a', newline='')
    wr = csv.writer(f)
    wr.writerow(contents)
    f.close()
    
def convert_xywha_to_8coords(xywha, is_clockwise=True):
    x, y, w, h, a = xywha
    angle = a if is_clockwise else -a

    lt_x, lt_y = -w / 2, -h / 2
    rt_x, rt_y = w / 2, - h/ 2
    rb_x, rb_y = w / 2, h / 2
    lb_x, lb_y = - w / 2, h / 2

    lt_x_ = lt_x * math.cos(angle) - lt_y * math.sin(angle) + x
    lt_y_ = lt_x * math.sin(angle) + lt_y * math.cos(angle) + y
    rt_x_ = rt_x * math.cos(angle) - rt_y * math.sin(angle) + x
    rt_y_ = rt_x * math.sin(angle) + rt_y * math.cos(angle) + y
    lb_x_ = lb_x * math.cos(angle) - lb_y * math.sin(angle) + x
    lb_y_ = lb_x * math.sin(angle) + lb_y * math.cos(angle) + y
    rb_x_ = rb_x * math.cos(angle) - rb_y * math.sin(angle) + x
    rb_y_ = rb_x * math.sin(angle) + rb_y * math.cos(angle) + y

    return [lt_x_, lt_y_, rt_x_, rt_y_, rb_x_, rb_y_, lb_x_, lb_y_]

In [5]:
all_imgs = glob('container_dataset/valid_images','*.png')
all_imgs.sort()
all_imgs[:5]
all_imgs=all_imgs

In [6]:
## inference 진행시에 test pipeline 거치기 때문에 test pipeline을 학습시에도 미리 set해두기 (5120,5120)
## 계획 ==> 이미지를 1024 patch로 640 stride로 inference
from mmrotate.apis import inference_detector_by_patches
import cv2

nn = 0
for img_nm in all_imgs:
    img = cv2.imread(img_nm)
    results=inference_detector_by_patches(model, img_nm, [1024], [512], [4.5, 5, 5.5], dict(type='nms_rotated', iou_threshold=0.1))
    File = img_nm.split('/')[-1][:8]
    for i in range(results.pred_instances.scores.shape[0]):
        if results.pred_instances.scores[i] >= 0.1:
            test_pos = list(convert_xywha_to_8coords(results.pred_instances.bboxes[i].cpu().numpy()))
            test_pos = np.array([[test_pos[0:2]], [test_pos[2:4]], [test_pos[4:6]], [test_pos[6:8]]], np.int32)
            img = cv2.polylines(img, [test_pos], True, (255, 255, 255), 2)
            cv2.imwrite('visualize_folder/'+File+'.png',img)
    nn+=1
    if nn%10==0:
        print(nn,' END!')



10  END!
20  END!
30  END!
40  END!
50  END!
60  END!
70  END!
80  END!
90  END!
100  END!
110  END!


KeyboardInterrupt: 